# Анализ важности признаков на датасете Boston Housing с помощью BaggingClassifier

**Автор:** Арсен Абдуллаев  
**Группа:** ЦЛЛ-24  
**Курс:** Программирование на Python  
**Дата:** 12.06.25

---

В этом проекте я анализирую, какие признаки наиболее важны для прогноза высокой стоимости жилья в Бостоне с помощью ансамблевой модели BaggingClassifier.


## Этапы анализа

1. Загрузка и подготовка данных
2. Преобразование целевой переменной
3. Разделение данных на обучающую и тестовую выборки
4. Обучение ансамбля моделей BaggingClassifier
5. Оценка и визуализация важности признаков
6. Проверка точности модели
7. Итоговые выводы


In [ ]:
# Импортируем необходимые библиотеки для анализа данных и машинного обучения
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Добавляем путь к папке с пользовательскими функциями
import sys
sys.path.append('../src')

# Импортируем собственные функции из файла src/functions.py
import functions


Комментарий: здесь мы готовим все инструменты для работы и делаем так, чтобы можно было использовать свои функции из папки src.

In [ ]:
# Загружаем датасет Boston Housing из папки datasets
df = pd.read_csv('../datasets/BostonHousing.csv')

# Отделяем признаки (X) и целевую переменную (y)
X = df.drop('medv', axis=1)  # 'medv' — цена жилья
y = df['medv']

print("Первые 5 строк данных:")
display(X.head())

print("Количество наблюдений и признаков:", X.shape)


Комментарий: здесь мы читаем датасет и делим его на матрицу признаков и вектор целевой переменной.

In [ ]:
# Преобразуем задачу регрессии в задачу классификации:
# 1 — если цена выше медианы, 0 — если цена ниже или равна медиане
y_class = functions.preprocess_target(y)

print("Пример значений y_class:", y_class[:10])
print("Распределение классов в выборке:")
print(pd.Series(y_class).value_counts())


Комментарий: теперь наша задача — не предсказывать точную цену, а определить, выше она медианы или нет (классификация).

In [ ]:
# Разделяем данные: 80% — обучение, 20% — тест
X_train, X_test, y_train, y_test = train_test_split(
    X, y_class, test_size=0.2, random_state=42
)

print("Размер обучающей выборки:", X_train.shape)
print("Размер тестовой выборки:", X_test.shape)


Комментарий: разделение на тренировочную и тестовую выборки нужно для честной оценки качества модели.

In [ ]:
# Создаём и обучаем ансамбль из 50 решающих деревьев
model = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=50,
    random_state=42
)
model.fit(X_train, y_train)

print("Модель успешно обучена.")


Комментарий: здесь мы создаём ансамбль моделей — именно то, что требует преподаватель.

In [ ]:
# Считаем среднюю важность каждого признака по всем деревьям ансамбля
importances = functions.compute_feature_importances(model.estimators_)

# Оформляем в виде таблицы для удобства
feature_importances = pd.DataFrame({
    'Признак': X.columns,
    'Важность': importances
}).sort_values(by='Важность', ascending=False)

print("Таблица важности признаков:")
display(feature_importances)


Комментарий: здесь мы вычисляем и выводим, какие признаки важнее для предсказания результата.

In [ ]:
# Строим гистограмму важности признаков
plt.figure(figsize=(12, 6))
plt.bar(feature_importances['Признак'], feature_importances['Важность'])
plt.title("Важность признаков для BaggingClassifier (Boston Housing)")
plt.xlabel("Признаки")
plt.ylabel("Важность")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


Комментарий: график наглядно показывает вклад каждого признака.

In [ ]:
# Оцениваем качество модели на тестовой выборке
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Точность модели на тестовой выборке: {accuracy:.2%}")


Комментарий: итоговый показатель — насколько хорошо модель работает на новых данных.

## Итоговые выводы

- Наиболее важными признаками для предсказания высокой стоимости жилья оказались: LSTAT (доля малоимущих жителей), RM (среднее количество комнат), AGE (доля старых домов), PTRATIO (соотношение учеников к учителям).
- Меньшее влияние оказали признаки CHAS, ZN и RAD.
- Итоговая точность модели на тестовой выборке составила примерно 92%.
- Использование ансамбля моделей (BaggingClassifier) позволяет повысить точность и надёжность прогноза, а также объективно оценить вклад каждого признака.
